<a href="https://colab.research.google.com/github/mars0925/100501/blob/master/media_pipe_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version
# !pip install mediapipe-model-maker
!pip install 'keras<3.0.0' mediapipe-model-maker

Python 3.10.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 M

In [2]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import image_classifier

import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
image_path = tf.keras.utils.get_file(
    'flower_photos.tgz',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

228813984/228813984 [==============================] - 2s 0us/step


In [4]:
print(image_path)
labels = []
for i in os.listdir(image_path):
  if os.path.isdir(os.path.join(image_path, i)):
    labels.append(i)
print(labels)

/root/.keras/datasets/flower_photos
['roses', 'tulips', 'dandelion', 'sunflowers', 'daisy']


In [5]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(image_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [6]:
data = image_classifier.Dataset.from_folder(image_path)
train_data, remaining_data = data.split(0.8)
test_data, validation_data = remaining_data.split(0.5)

In [7]:
spec = image_classifier.SupportedModels.MOBILENET_V2
hparams = image_classifier.HParams(export_dir="exported_model")
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

In [8]:
model = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________
None


Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BICUBIC...)` instead.


Epoch 1/10
1468/1468 [==============================] - 121s 79ms/step - loss: 1.7806 - accuracy: 0.2691 - val_loss: 1.3715 - val_accuracy: 0.4687
Epoch 2/10
1468/1468 [==============================] - 107s 73ms/step - loss: 1.2324 - accuracy: 0.5712 - val_loss: 0.9468 - val_accuracy: 0.7466
Epoch 3/10
1468/1468 [==============================] - 112s 77ms/step - loss: 0.9640 - accuracy: 0.7183 - val_loss: 0.8166 - val_accuracy: 0.8174
Epoch 4/10
1468/1468 [==============================] - 106s 72ms/step - loss: 0.8616 - accuracy: 0.7817 - val_loss: 0.7702 - val_accuracy: 0.8283
Epoch 5/10
1468/1468 [==============================] - 113s 77ms/step - loss: 0.8350 - accuracy: 0.7953 - val_loss: 0.7445 - val_accuracy: 0.8447
Epoch 6/10
1468/1468 [==============================] - 113s 77ms/step - loss: 0.8099 - accuracy: 0.8110 - val_loss: 0.7262 - val_accuracy: 0.8501
Epoch 7/10
1468/1468 [==============================] - 113s 77ms/step - loss: 0.8070 - accuracy: 0.8174 - val_loss: 0

In [9]:
loss, acc = model.evaluate(test_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

12/12 [==============================] - 16s 1s/step - loss: 0.6688 - accuracy: 0.8992
Test loss:0.6688261032104492, Test accuracy:0.8991825580596924


In [10]:
model.export_model()

In [11]:
!ls exported_model
files.download('exported_model/model.tflite')

checkpoint  metadata.json  model.tflite  summaries


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
hparams=image_classifier.HParams(epochs=15, export_dir="exported_model_2")

In [13]:
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)
options.model_options = image_classifier.ModelOptions(dropout_rate = 0.07)

In [14]:
model_2 = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________
None
Epoch 1/15
1468/1468 [==============================] - 122s 80ms/step - loss: 1.8921 - accuracy: 0.2248 - val_loss: 1.4481 - val_accuracy: 0.4387
Epoch 2/15
1468/1468 [==============================] - 107s 73ms/step - loss: 1.2322 - accuracy: 0.5593 - val_loss: 0.9350 - 

In [15]:
loss, accuracy = model_2.evaluate(test_data)

12/12 [==============================] - 22s 1s/step - loss: 0.6459 - accuracy: 0.8856


In [16]:
from mediapipe_model_maker import quantization

In [17]:
quantization_config = quantization.QuantizationConfig.for_int8(train_data)

In [18]:
model.export_model(model_name="model_int8.tflite", quantization_config=quantization_config)

In [19]:
ls -lh exported_model

total 12M
drwxr-xr-x 2 root root 4.0K Apr 16 06:33 checkpoint/
-rw-r--r-- 1 root root 1.7K Apr 16 07:13 metadata.json
-rw-r--r-- 1 root root 2.8M Apr 16 07:13 model_int8.tflite
-rw-r--r-- 1 root root 8.5M Apr 16 06:34 model.tflite
drwxr-xr-x 4 root root 4.0K Apr 16 06:14 summaries/
